In [24]:
###############
# Service version : 1.2.2
# Language        : Python/Jupyter

# Energy Loss Training
This recipe performs the training of a model that calculates an electrical transformer's the load factor, loss factor and energy loss in any given range of time. 


The data requirements are power quality data that contain timestamps, active power readings, and completion of the provided configuration file.

## Input
The data requirements for running the recipe for a given asset are Timestamps, and Active Power measurements:


| **asset_id** | **DeviceTimeStamp** | **Active Power (KW)** |
|--------------|---------------------|--------|
| 9010        | 2019-06-27T10:51    | 78.23312|
| 9010        | 2019-06-27T10:52    | 76.65175|
| 9010        | 2019-06-27T10:53    | 78.66275|
| 9010        | 2019-06-27T10:53    | 76.65175|

## Output


Load Factor: The ratio of the transformer's average load to its maximum rated capacity, indicating how efficiently it is utilized.

Loss Factor: A measure of the transformer's energy loss during operation, accounting for factors such as resistance, core, and stray load losses as per either Gustafson's or Buller & Woodrow's method.

Estimated Energy Loss: A calculated value representing the anticipated energy loss in the transformer during a given period, based on its load factor and peak power.


| asset_id | period | load_factor | loss_factor | energyloss (WH) |
|-------------|-------------|-------------|-------------|-------------|
|9010| 2019-06-25  | 0.353592    | 0.137005    | 345.253615 |
|9010| 2019-07-25  | 0.471900    | 0.237904    | 599.518190 |
|9010| 2019-08-24  | 0.512072    | 0.278126    | 700.876400 |
|9010| 2019-09-23  | 0.525415    | 0.292146    | 736.208374 |

# Import Libraries & Functions

In [2]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [3]:
# Print functions
def print_job_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    if "status" in status_data:
        print("The status of job {} is {}.".format(job_id, status_data['status']))
    else:
        print(status_data)
    
def print_all_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    print(status_data)


In [4]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))
    return summary_data

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            if "status" in summary:
                html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code


# Loading  Dataset

Loading the power quality dataset from the local project space

In [5]:
df = pd.read_csv("../data/training.csv")
df["Timestamp"] = df["Timestamp"].astype(str)
df.head(10)

,Timestamp,Active Power (KW)
0,2022-07-08 10:51:00,0.0
1,2022-07-08 10:52:00,0.0
2,2022-07-08 10:53:00,0.0
3,2022-07-08 10:53:00,0.0
4,2022-07-08 10:56:00,0.0
5,2022-07-08 10:56:00,0.0
6,2022-07-08 10:57:00,0.0
7,2022-07-08 11:01:00,0.0
8,2022-07-08 11:01:00,0.0
9,2022-07-08 11:10:00,0.0


# Energy Loss Deployment

## Defining Paths

In [6]:
# Define the file paths
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
config_file_path = "../config/parameter_config_three_kpis_loss_rate.yaml"
data_file_path = "../data/training.csv"

## Preparation of Configuration File

A requirement of the recipe is preparation of the configuration file for training. 


Information including column specification, loss factor calculation method, the option to use an a priori peak power are all options that can be configured.


Most important the "output" can be changed to "energy_loss", "load_factor" or "loss_factor"

For more detail for preparing the configuration file see the [documentation](https://github.ibm.com/srom/ModelFactory/blob/development/model_factory/recipes/electrical_transformer/docs/energyloss.md)

In [7]:
import requests

# Read the content of the config file
with open(config_file_path, 'r') as config_file:
    config_content = config_file.read()

# Display the content
print(config_content)

dataset_param:
  timestamp_col_name: 'Timestamp' # the dataset column specifies the sampling time
  power_col_name: 'Active Power (KW)' # the dataset column specifies the power of the electrical transformer
transformer:
  power_unit: 'KW'  # All the output is based on this unit
  transformer_rating: 100 # The maximum capacity to handle power without exceeding its design temperature
  peak_power: 104.45  # The peak value of the power transferred
estimate_loss_factor_method:  # estimate the power loss factor
  smooth_time_period: '1h'   # Smooth time range to allow the loss factor to be smoothed (medium is used), say '1h' or '2H'
  method: 'gustafson'  # Please choose  either 'gustafson' or 'buller_woodrow'
  gustafson_parameter: 1.912 # only used for 'gustafson' method, should be 1.912 all the time (only pertain to USA)
  buller_woodrow_parameter: 0.7 # only used for 'buller_woodrow' method, say 0.7, can be estimated (see README file)
energyloss_output:
  full_load_winding_loss: 3.5  # 

## POST Response

In [8]:
import requests

files = {
    "data_file": ("training.csv", open(data_file_path, 'rb')),
    "config_file": ("parameter_config.yaml", open(config_file_path, 'rb')),    
}

url = endpoint_url + "/recipe/electrical-transformer/kpi/energyloss"
post_response = requests.request("POST", url, headers={}, data={}, files=files)

In [15]:
post_r_json = post_response.json()

anomaly_service_jobId = None

if 'jobId' in post_r_json:
    anomaly_service_jobId = post_r_json['jobId']
    print ('submitted successfully job : ', post_r_json['jobId'])
else:
    print (post_r_json)
time.sleep(5)

{'job_id': 'e7f53652-ae73-413f-8d3d-67af26eea92b', 'message': 'Job e7f53652-ae73-413f-8d3d-67af26eea92b was submitted.', 'status': 'INITIALIZING'}


## GET Response  - Keep Refresh it for a while until seeing the STATUS changed to DONE

The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [16]:
log_url = endpoint_url + "/log/"
job_id = post_r_json['job_id']

In [17]:
print(log_url + job_id)

http://localhost:8000/ibm/modelfactory/service/log/e7f53652-ae73-413f-8d3d-67af26eea92b


## GET Response

In [18]:
job_id = post_r_json["job_id"]
# Get Status
print_job_status(job_id,endpoint_url)

The status of job e7f53652-ae73-413f-8d3d-67af26eea92b is DONE.


The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [19]:
# Get Status
print_job_status(job_id,endpoint_url)

The status of job e7f53652-ae73-413f-8d3d-67af26eea92b is DONE.


In [20]:
# Get selected details
json_data = print_job_details(job_id,endpoint_url)

# Get all details

print_all_status(job_id, endpoint_url)

{'job_id': 'e7f53652-ae73-413f-8d3d-67af26eea92b', 'status': 'DONE', 'summary': {'Deployment Instruction': 'For deployment, pass the parent_run_id as an argument in the deployment recipe', 'parent_run_id': '3bc240c4bed74d4d837e355399b9fdd5'}, 'detailed_summary': [{'run_id': 'd443512c9ba54cc5993beb2a56ba5480', 'experiment_id': '2', 'status': 'FINISHED', 'artifact_uri': 's3://testdataupload/2/d443512c9ba54cc5993beb2a56ba5480/artifacts', 'start_time': '2023-11-16T02:04:51.760000+00:00', 'end_time': '2023-11-16T02:04:58.062000+00:00', 'params.steps': 'energyloss', 'params.output_artifact_name': 'energyloss', 'params.spec_yaml': '', 'params.processed_data_path': '', 'params.storage_path': '', 'tags.mlflow.source.name': '/tmp/ray/session_2023-11-16_01-58-51_212131_1/runtime_resources/working_dir_files/_ray_pkg_b4ad6be3ad44c709', 'tags.mlflow.project.entryPoint': 'deployment_artifact', 'tags.mlflow.log-model.history': '[{"run_id": "d443512c9ba54cc5993beb2a56ba5480", "artifact_path": "energylo

### Saving details for deployment

In [21]:
json_data["detailed_summary"]

[{'run_id': 'd443512c9ba54cc5993beb2a56ba5480',
  'experiment_id': '2',
  'status': 'FINISHED',
  'artifact_uri': 's3://testdataupload/2/d443512c9ba54cc5993beb2a56ba5480/artifacts',
  'start_time': '2023-11-16T02:04:51.760000+00:00',
  'end_time': '2023-11-16T02:04:58.062000+00:00',
  'params.steps': 'energyloss',
  'params.output_artifact_name': 'energyloss',
  'params.spec_yaml': '',
  'params.processed_data_path': '',
  'params.storage_path': '',
  'tags.mlflow.source.name': '/tmp/ray/session_2023-11-16_01-58-51_212131_1/runtime_resources/working_dir_files/_ray_pkg_b4ad6be3ad44c709',
  'tags.mlflow.project.entryPoint': 'deployment_artifact',
  'tags.mlflow.log-model.history': '[{"run_id": "d443512c9ba54cc5993beb2a56ba5480", "artifact_path": "energyloss", "utc_time_created": "2023-11-16 02:04:54.775976", "flavors": {"python_function": {"model_path": "model.pkl", "predict_fn": "predict", "loader_module": "mlflow.sklearn", "python_version": "3.9.2", "env": {"conda": "conda.yaml", "virt

In [29]:
json_data["detailed_summary"]

[{'run_id': 'd443512c9ba54cc5993beb2a56ba5480',
  'experiment_id': '2',
  'status': 'FINISHED',
  'artifact_uri': 's3://testdataupload/2/d443512c9ba54cc5993beb2a56ba5480/artifacts',
  'start_time': '2023-11-16T02:04:51.760000+00:00',
  'end_time': '2023-11-16T02:04:58.062000+00:00',
  'params.steps': 'energyloss',
  'params.output_artifact_name': 'energyloss',
  'params.spec_yaml': '',
  'params.processed_data_path': '',
  'params.storage_path': '',
  'tags.mlflow.source.name': '/tmp/ray/session_2023-11-16_01-58-51_212131_1/runtime_resources/working_dir_files/_ray_pkg_b4ad6be3ad44c709',
  'tags.mlflow.project.entryPoint': 'deployment_artifact',
  'tags.mlflow.log-model.history': '[{"run_id": "d443512c9ba54cc5993beb2a56ba5480", "artifact_path": "energyloss", "utc_time_created": "2023-11-16 02:04:54.775976", "flavors": {"python_function": {"model_path": "model.pkl", "predict_fn": "predict", "loader_module": "mlflow.sklearn", "python_version": "3.9.2", "env": {"conda": "conda.yaml", "virt

In [25]:
import yaml
if 'status' in json_data:
    if json_data['status'] == 'DONE':
        for summary in json_data["detailed_summary"]:
            if summary["params.output_artifact_name"] != "":
                s3_uri = summary["artifact_uri"] + "/" + summary["params.output_artifact_name"]
            if summary['tags.artifact_name']!="":
                model_uri = "runs:/" + summary['run_id'] + "/" + summary['tags.artifact_name']
        output_data = {
            "model_uri" : model_uri,
            "train_job_id" : job_id,
            "s3_uri": s3_uri,
        }
        with open("../config/model_info.yml","w") as file:
            yaml.dump(output_data, file)
        output_data

In [26]:
output_data

{'model_uri': 'runs:/794a004e0fae4a28a80fe456ed08ae3d/energyloss',
 'train_job_id': 'e7f53652-ae73-413f-8d3d-67af26eea92b',
 's3_uri': 's3://testdataupload/2/d443512c9ba54cc5993beb2a56ba5480/artifacts/energyloss'}